In [105]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [106]:
root_data_dir = "../Datasets/NER/BiodivNER/"

dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [107]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="latin1")
  data = data.fillna(method="ffill")
  return data

In [108]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),                                                          
                                                        s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [109]:
data = loadData(train_csv_file_path)

In [110]:
val_data = loadData(val_csv_file_path)

In [111]:
test_data = loadData(test_csv_file_path)

In [112]:
VOCAB = list(set(list(data["Word"].values) + \
                 list(val_data["Word"].values) + \
                 list(test_data["Word"].values)))
VOCAB.append("ENDPAD")

n_words = len(VOCAB) #n_words includes all vocab from train and validation test.

tags = list(set(data["Tag"].values))

n_tags = len(tags)

In [113]:
getter = SentenceGetter(data)
sentences = getter.sentences
sent = getter.get_next()
print(sent)

[('bottom', 'O'), ('board', 'O'), ('debris', 'O'), (',', 'O'), ('frames', 'O'), (',', 'O'), ('landing', 'O'), ('platforms', 'O'), (')', 'O'), (',', 'O'), ('and', 'O'), ('isolates', 'O'), ('of', 'O'), ('microbes', 'O'), (',', 'O'), ('parasites', 'O'), ('and', 'O'), ('pathogens', 'O'), ('from', 'O'), ('honey', 'B-Organism'), ('bees', 'I-Organism'), ('.', 'O')]


In [114]:
getter_val = SentenceGetter(val_data)
sentences_val = getter_val.sentences
sent_val = getter_val.get_next()
print(sent_val)

[('However', 'O'), (',', 'O'), ('the', 'O'), ('lack', 'O'), ('of', 'O'), ('correlation', 'O'), ('between', 'O'), ('dung', 'O'), ('beetle', 'O'), ('community', 'B-Environment'), ('characteristics', 'O'), ('and', 'O'), ('dung', 'O'), ('removal', 'O'), ('highlights', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('further', 'O'), ('research', 'O'), ('into', 'O'), ('spatial', 'O'), ('variation', 'O'), ('in', 'O'), ('biodiversityÃ\x83Â¢Ã¢Â\x82Â¬Ã¢Â\x80Â\x9cecosystem', 'O'), ('function', 'O'), ('relationships', 'O'), ('and', 'O'), ('how', 'O'), ('the', 'O'), ('results', 'O'), ('of', 'O'), ('such', 'O'), ('studies', 'O'), ('are', 'O'), ('affected', 'O'), ('by', 'O'), ('methodological', 'O'), ('choices', 'O'), ('.', 'O')]


In [115]:
getter_test = SentenceGetter(test_data)
sentences_test = getter_test.sentences
sent_test = getter_test.get_next()
print(sent_test)

[('The', 'O'), ('primacy', 'O'), ('of', 'O'), ('either', 'O'), ('species', 'B-Quality'), ('or', 'O'), ('functional', 'O'), ('group', 'O'), ('richness', 'B-Quality'), ('effects', 'O'), ('depended', 'O'), ('on', 'O'), ('the', 'O'), ('sequence', 'O'), ('of', 'O'), ('testing', 'O'), ('these', 'O'), ('terms', 'O'), (',', 'O'), ('indicating', 'O'), ('that', 'O'), ('both', 'O'), ('aspects', 'O'), ('of', 'O'), ('richness', 'B-Quality'), ('were', 'O'), ('congruent', 'O'), ('and', 'O'), ('complementary', 'O'), ('to', 'O'), ('expected', 'O'), ('strong', 'O'), ('effects', 'O'), ('of', 'O'), ('legume', 'O'), ('presence', 'O'), ('and', 'O'), ('grass', 'B-Organism'), ('presence', 'O'), ('on', 'O'), ('plant', 'B-Organism'), ('chemical', 'B-Quality'), ('composition', 'I-Quality'), ('.', 'O')]


In [116]:
tag2id = {tag: id for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [117]:
tag2id_list = list(tag2id.items())

In [118]:
id2tag_list = list(id2tag.items())

In [119]:
def get_text_tags_lists(sentences):
  texts = []
  tags = []
  for sent in sentences: #list of tuples    
    sent_texts = []
    sent_tags = []  
    for tuple1 in sent:  
      sent_texts.append(tuple1[0])
      sent_tags.append(tuple1[1])

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [120]:
train_texts, train_tags = get_text_tags_lists(sentences)
val_texts, val_tags = get_text_tags_lists(sentences_val)
test_texts, test_tags = get_text_tags_lists(sentences_test)

In [121]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5TokenizerFast, AdamW, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [122]:
tokenizer = T5TokenizerFast.from_pretrained("t5-small")

In [128]:
label_all_tokens = False

In [147]:
def tokenize_and_align_labels(texts, tags):
    tokenized_inputs = tokenizer(texts, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [148]:
train_tokenized = tokenize_and_align_labels(train_texts, train_tags)
val_tokenized = tokenize_and_align_labels(val_texts, val_tags)
test_tokenized = tokenize_and_align_labels(test_texts, test_tags)

In [155]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shann\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

c:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [158]:
training_args = TrainingArguments(
    output_dir="./output_dir",  # Change this to your desired output directory
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=500,  # Save model every 500 steps
    evaluation_strategy="steps",
    eval_steps=250,  # Evaluate every 250 steps
    logging_steps=100,  # Log every 100 steps
    learning_rate=5e-5,
    save_total_limit=2,  # Only keep the last two models
    remove_unused_columns=False,  # Keep all columns in the dataset
    push_to_hub=False,  # Set to True if you want to push to the Hugging Face Model Hub
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`